# Combine results of simulations and make bar plots

Use plotly to display results from a list of supernovae model simulations processed with sspike.

## Create event totals data frame from list of models

Each processed model should have a `totals.txt` file with tabulated event rates by channel.

In [1]:
from os import listdir
from os.path import isfile

import pandas as pd
import plotly.express as px

from sspike.detectors import Detector
from sspike.beer import sort_channels

### Get file paths to tabulated results

In [2]:
# Path to sspike snowball directory.
snowball_dir = '/Users/joe/src/gitjoe/sspike/snowballs/'

# Simulation list and plot labels.
labels = {'F21-13': 'Fornax', 
          'N13-20-20-100': 'Nakazato-t1',
          'N13-20-20-200': 'Nakazato-t2',
          'N13-20-20-300': 'Nakazato-t3',
          'W20-13.0-1.23': 'Warren-a1',
          'W20-13.0-1.25': 'Warren-a2',
          'W20-13.0-1.27': 'Warren-a3'}

sn_names = list(labels.keys())

distance = 10.

# List for filepaths to totals files tabulated using sspike.beer.tab()
beer_tabs = []

# Folder containing simulation results for each model.
for name in sn_names:
    sn_dir = f"{snowball_dir}{name}/"
    # Results for distance of interest.
    for d in listdir(sn_dir):
        if f"{distance}kpc" in d:
            file_name = f"{sn_dir}{d}/totals.txt"
            if isfile(file_name):
                beer_tabs.append(file_name)
            else:
                print(f'File not found!\n{file_name}')

### Set detector for channel lists.

In [3]:
detector = Detector('kamland')
combos = {**sort_channels(detector.snow_channels),
          **sort_channels(detector.nc_channels)}

### Read each file into a data frame

In [4]:
# Columns for simulation type and events per channel.
# TODO: add other simulation properties for sorting.
column_names = ['model', 'sn_name', 'Channel', 'Counts', 'Progenitor']

events = pd.DataFrame(columns=column_names)
for i, sn in enumerate(sn_names):
    # One row for each simulation file.
    row = {'sn_name': sn, 'model': sn[:3], 'Progenitor': labels[sn]}

    
    # Counters for combining flavors by target.
    counts = {}
    for channel in combos:
        counts[channel] = 0.

    # Get results from sspike.beer.tab() for each file.
    with open(beer_tabs[i], 'r') as f:
        lines = f.readlines()

    # Sort by data type/processing method.
    skip = 0
    data_type = None
    nc_flav_count = 0

    # Check each line in file.
    for line in lines:
        # Skip blank line separators.
        if not line.strip():
            continue

        # Skip lines that are for checking, but not really of interest here.
        if skip:
            skip -= 1
            continue

        # Data types of interest: snow-smeared, sspike-nc.
        if len(line.split('-')) == 2:
            data_type = line.strip()

            # TODO: make this check target channel lengths.
            if data_type == 'snow-smeared':
                skip = 1
            if data_type == 'snow-unsmeared':
                skip = 23
            if data_type == 'sspike-basic':
                skip = 5
            if data_type == 'sspike-nc':
                skip = 33
            continue
        
        # Get channel name and number of events.
        try:
            channel, count = line.split(':')
        except:
            print(f"ERROR: {line}")
            continue

        # Add into counts dictionary for this file.
        for column in combos.keys():
            if channel in combos[column]:
                counts[column] += float(count)
                if column == 'p-nc':
                    nc_flav_count += 1
                break
        
        # Only looking at 200 keV energy cut for now (skipping 300 keV cut).
        if nc_flav_count == 4:
            break

    for channel in counts:
        row['Channel'] = channel
        row['Counts'] = counts[channel]
        events = events.append(row, ignore_index=True)

ERROR: 500 kev energy cut

ERROR: ------------------



In [22]:
bars = px.bar(events, x='Channel', y='Counts', 
              color='Progenitor', barmode='group', log_y=True)
bars.layout.bargap = 0.15
bars.layout.bargroupgap = 0.3
bars.layout.title = f'KamLAND events for 13 solar mass star at {distance} kpc'
bars.layout.font = dict(size=18, family="Times New Roman")
bars.write_image('../plots/m13.png', width=1100, height=500, scale=3)
bars